In [1]:
import requests, re
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time, random
from urllib.request import urlopen
import urllib.parse
import numpy as np
from IPython.display import display
import datetime
import Levenshtein as lev
from itertools import product

In [18]:
def get_soup_driver(base_url): # Retrieves soup from website using Selenium
#     driver = webdriver.Chrome()
    driver.get(base_url)
    time.sleep(10 + random.uniform(-1,1))
    r = driver.page_source
    
#     driver.quit()
    return bs(r, 'html.parser')  

In [29]:
def search_imdb(title): # get list of movies from IMDB with the title
    title = urllib.parse.quote(title, safe='~')
    url = f'https://www.imdb.com/search/title/?title={title}&title_type=feature'
    soup = get_soup_driver(url)
    titles = soup.find_all('h3')
    result = {}
#     titles = [title.text.strip() for title in soup.find_all('h3')]
#     print(titles)
    if len(titles) > 1:
        result['movie title'] = titles[0].find('a').text.strip()
        result['movie date'] = titles[0].find(class_='lister-item-year').text[1:-1]
    else:
        result['movie title'] = False
        result['movie date'] = False

    return len(titles) > 1, result

In [41]:
def get_page_price_date_movie(title):
    base_url = 'https://www.google.com/search?q='
    title_url = urllib.parse.quote(title + ' book', safe='~')
    results = {}
    # do google search for book and get price
    print(title)
    # use selenium because page number doesn't always come up using requests module.
    try:
        soup = get_soup_driver(base_url + title_url)
        price = float(soup.find(text = 'Kindle Store').next.text.split()[0].replace('$', ''))
        price = float(price)
    except: 
        try: 
            price = float(soup.find(text = 'Barnes & Noble').next.text.split()[0].replace('$', ''))
            price = float(price)
        except:
            try:
                price = soup.find(text=re.compile('\$[0-9]+'))[1:]
                price = float(price)
            except Exception as e:
                price = -1
  
    # get movie info
    
    movie_info = search_imdb(title)
    
    if movie_info[0]:
        movie_title = movie_info[1]['movie title']
        movie_year = movie_info[1]['movie date']
    else:
        movie_title = 'N/A'
        movie_year = 'N/A'  
    
    results['price'] = str(price)
    results['is_movie'] = movie_info[0]
    results['movie title'] = movie_title
    results['movie year'] = str(movie_year)
    
    return results

In [43]:
def put_page_imdb(path):
    global driver
    driver = webdriver.Chrome()
    df = pd.read_csv('Figures/merged_df_lev_cleaned.csv', index_col=0)
    df[['price', 'movie title', 'movie year']] = df[['price', 'movie title', 'movie year']].astype('string')
    list_of_indices = list(df[(df.price == '1') | (df.price == "'NoneType' object is not subscriptable") | (df.price == '0')| (df['movie title'] == '1')].index)
    for index in list_of_indices:
        try:
            if df.loc[index, 'title'] == '-1': raise Exception("Title is -1") 
            search_results = get_page_price_date_movie(df.loc[index, 'title'])
            df.loc[index, 'price'] = search_results['price']
            df.loc[index, 'is_movie'] = search_results['is_movie']
            df.loc[index, 'movie title'] = search_results['movie title']
            df.loc[index, 'movie year'] = search_results['movie year']
        except Exception as e:
            df.loc[index, 'price'] = '-1'
            df.loc[index, 'is_movie'] = '-1'
            df.loc[index, 'movie title'] = '-1'
            df.loc[index, 'movie year'] = '-1'
            print(e)
            driver.quit()
            time.sleep(10)
            driver = webdriver.Chrome()
            continue
        
        df.to_csv(path)
        
        

In [44]:
put_page_imdb('Figures/merged_df_lev_cleaned.csv')

Title is -1
Amazing Spider-Man by Nick Spencer, Vol. 4: Hunted
The Bean Trees
The Bean Trees
The Seven Husbands of Evelyn Hugo
The Seven Husbands of Evelyn Hugo
Lover Eternal
Lover Eternal
Words of Radiance
Words of Radiance
250 A.D.: Una historia de ayer, que podría suceder hoy
The Book of Malachi
One Day in the Life of Ivan Denisovich
Cress
Cress
The Knife of Never Letting Go
Left Behind
Left Behind
Poison Study
Poison Study
The Haunting of Hill House
The Haunting of Hill House
Foucault's Pendulum
The Memory Keeper's Daughter
The Memory Keeper's Daughter
The Eyre Affair
The Eyre Affair
If on a Winter's Night a Traveler
If on a Winter's Night a Traveler
Where She Went
Where She Went
1Q84
1Q84
Title is -1
Cliffsnotes on Dante's Divine Comedy: Inferno (Cliffs Notes)
Seabiscuit: An American Legend
Seabiscuit: An American Legend
A Brief History of Time
A Brief History of Time
Shadow and Bone
Shadow and Bone
Shadowfever
Momma Gone
Momma Gone
A Wizard of Earthsea
A Wizard of Earthsea
The Le

Their Eyes Were Watching God
The Ultimate Hitchhiker's Guide to the Galaxy
Eat, Pray, Love: One Woman's Search for Everything Across Italy, India and Indonesia
Eat, Pray, Love: One Woman's Search for Everything Across Italy, India and Indonesia
Clockwork Princess
Clockwork Princess
The Battle of the Labyrinth
The Battle of the Labyrinth
Goodnight Moon
Goodnight Moon
The Sea of Monsters
The Sea of Monsters
The Prophet
The Prophet
The Mists of Avalon
The Mists of Avalon
Madame Bovary, The 100 Greatest Books Ever Written
Madame Bovary, The 100 Greatest Books Ever Written
A Storm of Swords
A Storm of Swords
Throne of Glass
Throne of Glass
Wizard and Glass
Wizard and Glass
The Brief Wondrous Life of Oscar Wao
The Brief Wondrous Life of Oscar Wao
Beach Music
Beach Music
Thus Spoke Zarathustra
Thus Spoke Zarathustra
The Runaway Jury
The Runaway Jury
Beauty and the Billionaire: The Wedding
Beauty and the Billionaire: The Wedding
Light in August
Light in August
Faefever
From Dead to Worse
The D

KeyboardInterrupt: 

In [34]:
df = pd.read_csv('Figures/merged_df_lev_cleaned.csv', index_col=0)

In [11]:
df[['price', 'movie title', 'movie year']] = df[['price', 'movie title', 'movie year']].astype('string')

In [36]:
test = df[(df.price == '1') | (df.price == '-1') | (df.price == "'NoneType' object is not subscriptable") | (df.price == '0')| (df['movie title'] == '1')]

In [37]:
test[['price', 'movie title', 'movie year']].astype('string')

,price,movie title,movie year
8,1,1,1
10,1,1,1
16,7.99,1,1
18,7.99,1,1
20,13.99,1,1
...,...,...,...
5257,1,1,1
5258,1,1,1
5260,1,1,1
5266,1,1,1


In [40]:
df.dtypes

id_number                  float64
title                       object
author                      object
author_url                  object
stars                       object
number of ratings           object
url                         object
is_series                   object
year_published             float64
price                       object
number of pages              int64
is_movie                    object
movie title                 object
movie year                  object
overdrive_url               object
media                       object
Overdrive title             object
Wait time                   object
Library copies              object
People waiting in total     object
People waiting per copy     object
Genres                      object
overdrive_author            object
author_stars               float64
author_ratings             float64
author_num_reviews         float64
author_distinct_works      float64
LevScore                   float64
LevScore_updated    